In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import math
import random
import numpy as np
from scipy.optimize import minimize

In [11]:
def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    # circ.cz(qubits[0], qubits[1])
    # circ.cz(qubits[2], qubits[0])

    # for iz in range (0, len(qubits)):
    #     circ.ry(parameters[1][iz], qubits[iz])

    # circ.cz(qubits[1], qubits[2])
    # circ.cz(qubits[2], qubits[0])

    # for iz in range (0, len(qubits)):
    #     circ.ry(parameters[2][iz], qubits[iz])

circ = QuantumCircuit(1)
apply_fixed_ansatz([0], [ [1]  ])
circ.draw()

┌───────┐
q: ┤ Ry(1) ├
   └───────┘

In [12]:
# Creates the Hadamard test

def had_test(gate_type, qubits, auxiliary_index, parameters):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters)
    for first in range(0, len(gate_type)):
        for ie in range (0, len(gate_type[0])):
            if (gate_type[first][ie] == 1):
                circ.cz(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 2):
                # print(2)
                circ.cx(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 3):
                circ.cy(auxiliary_index, qubits[ie])

    # for ie in range (0, len(gate_type[1])):
    #     if (gate_type[1][ie] == 1):
    #         circ.cz(auxiliary_index, qubits[ie])
    #     elif (gate_type[0][ie] == 2):
    #         circ.cx(auxiliary_index, qubits[ie])
    #     elif (gate_type[0][ie] == 3):
    #         circ.cy(auxiliary_index, qubits[ie])
    
    circ.h(auxiliary_index)
    
circ = QuantumCircuit(2)
had_test([ [1] ], [1], 0, [ [1] ])
circ.draw()

circ = QuantumCircuit(3)
had_test([ [0,1], [1,0] ], [1, 2], 0, [ [1, 1],])
circ.draw()

┌───┐        ┌───┐
q_0: ──┤ H ├───■──■─┤ H ├
     ┌─┴───┴─┐ │  │ └───┘
q_1: ┤ Ry(1) ├─┼──■──────
     ├───────┤ │         
q_2: ┤ Ry(1) ├─■─────────
     └───────┘

In [14]:
# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    # circ.ccx(auxiliary, qubits[1], 2)
    # circ.cz(qubits[0], 0)
    circ.ccx(auxiliary, qubits[0], 2)

    # circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[0], 2)
    # circ.ccx(auxiliary, qubits[0], 4)

    circ.ccx(auxiliary, qubits[0], 2)

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    # for i in range (0, len(qubits)):
    #     circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    # circ.ccx(auxiliary, qubits[2], 2)
    # circ.cz(qubits[1], 2)
    # circ.ccx(auxiliary, qubits[2], 2)

    # circ.ccx(auxiliary, qubits[0], 4)
    # circ.cz(qubits[2], 4)
    # circ.ccx(auxiliary, qubits[0], 4)

    # for i in range (0, len(qubits)):
    #     circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

q_reg = QuantumRegister(3)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1], [ [1] ], 0, q_reg)
circ.draw()

q137_0: ────■──────■───────■──────■────
        ┌───┴───┐  │       │  ┌───┴───┐
q137_1: ┤ Ry(1) ├──■───■───■──┤ Ry(1) ├
        └───────┘┌─┴─┐ │ ┌─┴─┐└───────┘
q137_2: ─────────┤ X ├─■─┤ X ├─────────
                 └───┘   └───┘

In [15]:
def control_b(auxiliary, qubits):

    for ia in qubits:
        circ.cz(auxiliary, ia)

circ = QuantumCircuit(2)
control_b(0, [1])
circ.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [16]:
# Create the controlled Hadamard test, for calculating <psi|psi>

def special_had_test(gate_type, qubits, auxiliary_index, parameters, reg):

    circ.h(auxiliary_index)

    control_fixed_ansatz(qubits, parameters, auxiliary_index, reg)

    for ty in range (0, len(gate_type)):
        if (gate_type[ty] == 1):
            circ.cz(auxiliary_index, qubits[ty])
        # elif (gate_type[0][ty] == 2):
        #     circ.cx(auxiliary_index, qubits[ty])
        # elif (gate_type[0][ty] == 3):
        #     circ.cy(auxiliary_index, qubits[ty])


    control_b(auxiliary_index, qubits)
    
    circ.h(auxiliary_index)

q_reg = QuantumRegister(3)
circ = QuantumCircuit(q_reg)
special_had_test([ [ 1] ], [1], 0, [ [1] ], q_reg)
circ.draw()

┌───┐                                  ┌───┐
q138_0: ┤ H ├────■──────■───────■──────■─────■─┤ H ├
        └───┘┌───┴───┐  │       │  ┌───┴───┐ │ └───┘
q138_1: ─────┤ Ry(1) ├──■───■───■──┤ Ry(1) ├─■──────
             └───────┘┌─┴─┐ │ ┌─┴─┐└───────┘        
q138_2: ──────────────┤ X ├─■─┤ X ├─────────────────
                      └───┘   └───┘

In [17]:
# coefficient_set = [0.55, 0.45]
# gate_set = [ [0, 0, 0], [0, 0, 1] ]

coefficient_set = [0.55, 0.45]
gate_set = [ [0], [1] ]

# Implements the entire cost function on the quantum circuit

def calculate_cost_function(parameters):
    
    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:1]]

    print(parameters)

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(3)
            qc = ClassicalRegister(3)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1], 0, parameters)

            circ.save_statevector()
            t_circ = transpile(circ, backend)
            # qobj = assemble(t_circ)
            job = backend.run(t_circ)

            result = job.result()
            outputstate = np.real(result.get_statevector(circ, decimals=100))
            o = outputstate

            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = o[l]**2
                    m_sum+=n

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(3)
                qc = ClassicalRegister(3)
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test([gate_set[i]], [1], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1], 0, parameters, qctl)

                circ.save_statevector()    
                t_circ = transpile(circ, backend)
                # qobj = assemble(t_circ)
                job = backend.run(t_circ)

                result = job.result()
                outputstate = np.real(result.get_statevector(circ, decimals=100))
                o = outputstate

                m_sum = 0
                for l in range (0, len(o)):
                    if (l%2 == 1):
                        n = o[l]**2
                        m_sum+=n
                mult = mult*(1-(2*m_sum))

            overall_sum_2+=multiply*mult

    print(overall_sum_1, overall_sum_2)
            
    # print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [18]:

out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/10000 for i in range(0, 1)], method="COBYLA", options={'maxiter':200})

# out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/10000 for i in range(0, 1)], method="COBYLA", options={'maxiter':200})

print(out)

out_f = [out['x'][0:1]]




[array([0.1635])]
0.99339849389485 1.0
[array([1.1635])]
0.701083498166761 1.0
[array([2.1635])]
0.22849023850953404 1.0
[array([3.1635])]
0.010118778376542903 1.0
[array([4.1635])]
0.2467388311424747 1.0
[array([3.6635])]
0.0758994098181131 1.0
[array([2.9135])]
0.02282076939451544 1.0
[array([3.2885])]
0.015331888031947127 1.0
[array([3.101])]
0.010407765476181496 1.0
[array([3.19475])]
0.010699196943783296 1.0
[array([3.147875])]
0.010009768267285668 1.0
[array([3.13225])]
0.010021602923949102 1.0
[array([3.1556875])]
0.010049168698078315 1.0
[array([3.14396875])]
0.010001397343294971 1.0
[array([3.1400625])]
0.010000579488964045 1.0
[array([3.13810937])]
0.010003002971322783 1.0
[array([3.14103906])]
0.010000075849614376 1.0
[array([3.14201562])]
0.010000044278940856 1.0
[array([3.14250391])]
0.010000205519384797 1.0
[array([3.14177148])]
0.01000000791516148 1.0
[array([3.14152734])]
0.010000001055680718 1.0
[array([3.1412832])]
0.01000002370049835 1.0
[array([3.14142734])]
0.01000

In [10]:
out_f

[array([3.14165196, 0.1672352 ]), array([0.14809874, 0.02109167])]

In [12]:
circ = QuantumCircuit(1, 1)
apply_fixed_ansatz([0], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
print(circ)

job = backend.run(t_circ, shots=8192)

result = job.result()
print(result.get_counts())
o = result.get_statevector(circ, decimals=10)
print(o)

a1 = coefficient_set[1]*np.array([ [1,0], [0,-1] ])
a2 = coefficient_set[0]*np.array([ [1,0], [0,1]])
# a2x = coefficient_set[0]*np.array([ [0,1,0,0,0,0,0,0], [1,0,0,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,1,0] ])

a3 = np.add(a2, a1)

b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

# b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o))))))



     ┌────────────┐ statevector 
  q: ┤ Ry(3.1417) ├──────░──────
     └────────────┘      ░      
c: 1/═══════════════════════════
                                
{'0': 8192}
Statevector([-2.96537e-05+0.j,  1.00000e+00+0.j],
            dims=(2,))
(0.7068970667826803+0j)
